##   从数据中抽取合适的特征

In [1]:
val rawData = sc.textFile("../data/kaggle/train_noheader.tsv")
val records = rawData.map(line => line.split("\t"))
records.first()

Array("http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html", "4042", "{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees ...

In [2]:
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.Vectors
val data = records.map{
    r =>
    val trimmed = r.map(_.replaceAll("\"",""))
    val label = trimmed(r.size -1).toInt
    val features = trimmed.slice(4,r.size-1).map(d => if(d=="?") 0.0 else d.toDouble)
    LabeledPoint(label, Vectors.dense(features))
}

In [4]:
data.cache
val numData = data.count
numData

7395

In [14]:
val nbData = records.map{
    r =>
    val trimmed = r.map(_.replaceAll("\"",""))
    val label = trimmed(r.size -1).toInt
    val features = trimmed.slice(4,r.size-1).map(d => if(d=="?") 0.0 else d.toDouble).map(d => if (d<0) 0.0 else d )
    LabeledPoint(label, Vectors.dense(features))
}

## 训练分类模型

In [52]:
import org.apache.spark.mllib.classification.LogisticRegressionWithSGD
import org.apache.spark.mllib.classification.SVMWithSGD
import org.apache.spark.mllib.classification.NaiveBayes
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.configuration.Algo
import org.apache.spark.mllib.tree.impurity.Entropy
val numIterations = 10
val maxTreeDepth = 5

In [9]:
val lrModel = LogisticRegressionWithSGD.train(data, numIterations)
lrModel

0.5

In [10]:
val svmModel = SVMWithSGD.train(data, numIterations)
svmModel

0.0

In [15]:
val nbModel = NaiveBayes.train(nbData)
nbModel

org.apache.spark.mllib.classification.NaiveBayesModel@72b0e188

In [16]:
val dtModel = DecisionTree.train(data, Algo.Classification, Entropy, maxTreeDepth)
dtModel

DecisionTreeModel classifier of depth 5 with 61 nodes

## 使用分类模型

In [17]:
val dataPoint = data.first
val prediction = lrModel.predict(dataPoint.features)
prediction

1.0

In [18]:
val trueLabel = dataPoint.label
trueLabel

0.0

In [19]:
val predictions = lrModel.predict(data.map(lp => lp.features))
predictions.take(5)

Array(1.0, 1.0, 1.0, 1.0, 1.0)

## 评估分类模型的性能

### 预测正确率和错误率

In [20]:
val lrTotalCorrect = data.map{ point =>
    if ( lrModel.predict(point.features) == point.label) 1 else 0
}.sum
val lrAccuracy = lrTotalCorrect / data.count
lrAccuracy

0.5146720757268425

In [22]:
val svmTotalCorrect = data.map{point=>
    if(svmModel.predict(point.features) == point.label) 1 else 0
}.sum
val nbTotalCorrect = nbData.map{ point =>
    if(nbModel.predict(point.features) == point.label) 1 else 0
}.sum

In [23]:
val dtTotalCorrect = data.map{ point =>
    val score = dtModel.predict(point.features)
    val predicted = if(score > 0.5) 1 else 0
    if( predicted == point.label ) 1 else 0
}.sum

In [24]:
val svmAccuracy = svmTotalCorrect / numData
svmAccuracy

0.5146720757268425

In [25]:
val nbAccuracy = nbTotalCorrect / numData
nbAccuracy

0.5803921568627451

In [26]:
val dtAccuracy = dtTotalCorrect / numData
dtAccuracy

0.6482758620689655

### 准确率和召回率

### ROC曲线和AUC

In [27]:
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
val metrics = Seq(lrModel,svmModel).map{ model =>
    val scoreAndLabels = data.map{ point =>
        (model.predict(point.features),point.label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (model.getClass.getSimpleName,metrics.areaUnderPR,metrics.areaUnderROC)
}

In [29]:
val nbMetrics = Seq(nbModel).map{ model =>
    val scoreAndLabels = nbData.map{ point =>
        val score = model.predict(point.features)
        (if (score > 0.5) 1.0 else 0.0, point.label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (model.getClass.getSimpleName, metrics.areaUnderPR,metrics.areaUnderROC)
}

In [30]:
val dtMetrics = Seq(dtModel).map{ model =>
    val scoreAndLabels = data.map{ point =>
        val score = model.predict(point.features)
        (if (score > 0.5) 1.0 else 0.0, point.label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (model.getClass.getSimpleName,metrics.areaUnderPR,metrics.areaUnderROC)
}

In [31]:
val allMetrics = metrics ++ nbMetrics ++ dtMetrics
allMetrics.foreach{ case (m, pr, roc) =>
    println(f"$m, Area under PR: ${pr * 100.0}%2.4f%%, Area under ROC: ${roc * 100.0}%2.4f%%")
}

LogisticRegressionModel, Area under PR: 75.6759%, Area under ROC: 50.1418%
SVMModel, Area under PR: 75.6759%, Area under ROC: 50.1418%
NaiveBayesModel, Area under PR: 68.0851%, Area under ROC: 58.3559%
DecisionTreeModel, Area under PR: 74.3081%, Area under ROC: 64.8837%


## 改进模型性能以及参数调优

### 特征标准化

In [32]:
 import org.apache.spark.mllib.linalg.distributed.RowMatrix
 val vectors = data.map( lp => lp.features )
 val matrix = new RowMatrix(vectors)
 val matrixSummary = matrix.computeColumnSummaryStatistics()
 println(matrixSummary.mean)

[0.4122580529952672,2.761823191986608,0.4682304732861389,0.21407992638350232,0.09206236071899916,0.04926216043908053,2.255103452212041,-0.10375042752143335,0.0,0.05642274498417851,0.02123056118999324,0.23377817665490194,0.2757090373659236,0.615551048005409,0.6603110209601082,30.07707910750513,0.03975659229208925,5716.598242055447,178.75456389452353,4.960649087221096,0.17286405047031742,0.10122079189276552]


In [33]:
println(matrixSummary.min)

[0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.045564223,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0]


In [34]:
println(matrixSummary.max)

[0.999426,363.0,1.0,1.0,0.980392157,0.980392157,21.0,0.25,0.0,0.444444444,1.0,0.716883117,113.3333333,1.0,1.0,100.0,1.0,207952.0,4997.0,22.0,1.0,1.0]


In [46]:
println(matrixSummary.variance)

[0.10974244167559001,74.30082476809639,0.04126316989120241,0.02153343633200108,0.009211817450882448,0.005274933469767946,32.53918714591821,0.09396988697611545,0.0,0.0017177410346628928,0.020782634824610638,0.0027548394224293036,3.683788919674426,0.2366799607085986,0.22433071201674218,415.8785589543846,0.03818116876739597,7.877330081138463E7,32208.116247426184,10.45300904576431,0.03359363403832393,0.006277532884214705]


In [35]:
println(matrixSummary.numNonzeros)

[5053.0,7354.0,7172.0,6821.0,6160.0,5128.0,7350.0,1257.0,0.0,7362.0,157.0,7395.0,7355.0,4552.0,4883.0,7347.0,294.0,7378.0,7395.0,6782.0,6868.0,7235.0]


In [43]:
import org.apache.spark.mllib.feature.StandardScaler
val scaler = new StandardScaler(withMean = true, withStd = true).fit(vectors)
val scaledData = data.map(lp => LabeledPoint(lp.label,scaler.transform(lp.features)))

[0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]


In [45]:
println( data.first.features )

[0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]


In [44]:
println( scaledData.first.features )

[1.137647336497678,-0.08193557169294771,1.0251398128933331,-0.05586356442541689,-0.4688932531289357,-0.3543053263079386,-0.3175352172363148,0.3384507982396541,0.0,0.828822173315322,-0.14726894334628504,0.22963982357813484,-0.14162596909880876,0.7902380499177364,0.7171947294529865,-0.29799681649642257,-0.2034625779299476,-0.03296720969690391,-0.04878112975579913,0.9400699751165439,-0.10869848852526258,-0.2788207823137022]


In [49]:
println( ( 0.789131 - 0.4122580529952672 ) / math.sqrt( 0.10974244167559001 ) )

1.137647336497678


In [55]:
val lrModelScaled = LogisticRegressionWithSGD.train( scaledData, numIterations )
val lrTotalCorrectScaled = scaledData.map { point =>
    if (lrModelScaled.predict(point.features) == point.label) 1 else 0
}.sum
val lrAccuracyScaled = lrTotalCorrectScaled / numData
val lrPredictionsVsTrue = scaledData.map{ point =>
    (lrModelScaled.predict(point.features),point.label)
}
val lrMetricsScaled = new BinaryClassificationMetrics(lrPredictionsVsTrue)
val lrPr = lrMetricsScaled.areaUnderPR
val lrRoc = lrMetricsScaled.areaUnderROC
println(f"${lrModelScaled.getClass.getSimpleName}\nAccuracy:${lrAccuracyScaled * 100}%2.4f%%\nArea under PR: ${lrPr * 100.0}%2.4f%%\nArea under ROC: ${lrRoc * 100.0}%2.4f%%")

LogisticRegressionModel
Accuracy:62.0419%
Area under PR: 72.7254%
Area under ROC: 61.9663%


### 其它特征

In [56]:
val categories = records.map(r => r(3)).distinct.collect.zipWithIndex.toMap
val numCategories = categories.size
println(categories)

Map("weather" -> 0, "sports" -> 6, "unknown" -> 4, "computer_internet" -> 12, "?" -> 11, "culture_politics" -> 3, "religion" -> 8, "recreation" -> 2, "arts_entertainment" -> 9, "health" -> 5, "law_crime" -> 10, "gaming" -> 13, "business" -> 1, "science_technology" -> 7)


In [57]:
println(numCategories)

14


In [58]:
val dataCategories = records.map{ r =>
    val trimmed = r.map(_.replaceAll("\"",""))
    val label = trimmed(r.size -1).toInt
    val categoryIdx = categories(r(3))
    val categoryFeatures = Array.ofDim[Double](numCategories)
    categoryFeatures(categoryIdx) = 1.0
    val otherFeatures = trimmed.slice(4,r.size -1).map( d => if(d == "?") 0.0 else d.toDouble)
    val features = categoryFeatures ++ otherFeatures
    LabeledPoint( label, Vectors.dense(features))
}
println( dataCategories.first )

(0.0,[0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575])


In [60]:
val scalerCats = new StandardScaler(withMean = true, withStd = true).fit(dataCategories.map(lp => lp.features))
val scaledDataCats = dataCategories.map(lp => LabeledPoint(lp.label,scalerCats.transform(lp.features)))

In [61]:
println( dataCategories.first.features )

[0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]


In [62]:
println( scaledDataCats.first.features )

[-0.02326210589837061,2.7207366564548514,-0.4464212047941535,-0.22052688457880879,-0.028494000387023734,-0.2709990696925828,-0.23272797709480803,-0.2016540523193296,-0.09914991930875496,-0.38181322324318134,-0.06487757239262681,-0.6807527904251456,-0.20418221057887365,-0.10189469097220732,1.137647336497678,-0.08193557169294771,1.0251398128933331,-0.05586356442541689,-0.4688932531289357,-0.3543053263079386,-0.3175352172363148,0.3384507982396541,0.0,0.828822173315322,-0.14726894334628504,0.22963982357813484,-0.14162596909880876,0.7902380499177364,0.7171947294529865,-0.29799681649642257,-0.2034625779299476,-0.03296720969690391,-0.04878112975579913,0.9400699751165439,-0.10869848852526258,-0.2788207823137022]


In [64]:
val lrModelScaledCats = LogisticRegressionWithSGD.train( scaledDataCats, numIterations )
val lrTotalCorrectScaledCats = scaledDataCats.map { point =>
    if (lrModelScaledCats.predict(point.features) == point.label) 1 else 0
}.sum
val lrAccuracyScaledCats = lrTotalCorrectScaledCats / numData
val lrPredictionsVsTrueCats = scaledDataCats.map{ point =>
    (lrModelScaledCats.predict(point.features),point.label)
}
val lrMetricsScaledCats = new BinaryClassificationMetrics(lrPredictionsVsTrueCats)
val lrPrCats = lrMetricsScaledCats.areaUnderPR
val lrRocCats = lrMetricsScaledCats.areaUnderROC
println(f"${lrModelScaledCats.getClass.getSimpleName}\nAccuracy:${lrAccuracyScaledCats * 100}%2.4f%%\nArea under PR: ${lrPrCats * 100.0}%2.4f%%\nArea under ROC: ${lrRocCats * 100.0}%2.4f%%")

LogisticRegressionModel
Accuracy:66.5720%
Area under PR: 75.7964%
Area under ROC: 66.5483%


In [65]:
val dataNB = records.map{ r =>
    val trimmed = r.map(_.replaceAll("\"",""))
    val label = trimmed( r.size - 1 ).toInt
    val categoryIdx = categories( r(3) )
    val categoryFeatures = Array.ofDim[Double](numCategories)
    categoryFeatures(categoryIdx) = 1.0
    LabeledPoint( label, Vectors.dense(categoryFeatures))
}

In [66]:
val nbModelCats = NaiveBayes.train(dataNB)
val nbTotalCorrectCats = dataNB.map{ point =>
    if( nbModelCats.predict(point.features) == point.label) 1 else 0
}.sum
val nbAccuracyCats = nbTotalCorrectCats / numData
val nbPredictionsVsTrueCats = dataNB.map{ point =>
    (nbModelCats.predict(point.features), point.label)
}
val nbMetricsCats = new BinaryClassificationMetrics(nbPredictionsVsTrueCats)
val nbPrCats = nbMetricsCats.areaUnderPR
val nbRocCats = nbMetricsCats.areaUnderROC
println(f"${nbModelCats.getClass.getSimpleName}\nAccuracy:${nbAccuracyCats * 100}%2.4f%%\nArea under PR: ${nbPrCats * 100.0}%2.4f%%\nArea under ROC: ${nbRocCats * 100.0}%2.4f%%")

NaiveBayesModel
Accuracy:60.9601%
Area under PR: 74.0522%
Area under ROC: 60.5138%


### 模型参数调优

In [69]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.optimization.Updater
import org.apache.spark.mllib.optimization.SimpleUpdater
import org.apache.spark.mllib.optimization.L1Updater
import org.apache.spark.mllib.optimization.SquaredL2Updater
import org.apache.spark.mllib.classification.ClassificationModel

In [70]:
def trainWithParams( input: RDD[LabeledPoint], regParam: Double, numIterations: Int, updater: Updater, stepSize: Double) = {
    val lr = new LogisticRegressionWithSGD
    lr.optimizer.setNumIterations(numIterations).setUpdater(updater).setRegParam(regParam).setStepSize(stepSize)
    lr.run(input)
}

In [71]:
def createMetrics(label: String, data: RDD[LabeledPoint],model:ClassificationModel) = {
    val scoreAndLabels = data.map{ point =>
        (model.predict(point.features), point.label)
    }
    val metrics = new BinaryClassificationMetrics( scoreAndLabels )
    ( label, metrics.areaUnderROC)
}

In [72]:
scaledDataCats.cache

MapPartitionsRDD[342] at map at <console>:64

In [73]:
val iterResults = Seq(1 ,5 ,10, 50).map { param =>
    val model = trainWithParams(scaledDataCats, 0.0, param, new SimpleUpdater, 1.0)
    createMetrics(s"$param iterations", scaledDataCats, model)
}
iterResults.foreach{ case (param, auc) => println(f"$param, AUC = ${auc * 100}%2.2f%%") }

1 iterations, AUC = 64.95%
5 iterations, AUC = 66.62%
10 iterations, AUC = 66.55%
50 iterations, AUC = 66.81%


In [76]:
val stepResults = Seq(0.001, 0.01, 0.1, 10.0).map{ param =>
    val model = trainWithParams(scaledDataCats, 0.0, numIterations, new SimpleUpdater, param)
    createMetrics(s"$param step size", scaledDataCats, model)
}
stepResults.foreach{ case ( param, auc ) => println( f"$param, AUC = ${auc * 100}%2.2f%%")}

0.001 step size, AUC = 64.97%
0.01 step size, AUC = 64.96%
0.1 step size, AUC = 65.52%
10.0 step size, AUC = 61.92%


In [79]:
val regResults = Seq(0.001, 0.01, 0.1, 10.0).map{ param =>
    val model = trainWithParams(scaledDataCats, param, numIterations,new SquaredL2Updater,1.0)
    createMetrics(s"$param L2 regularization parameter", scaledDataCats, model)
}
regResults.foreach{ case (param, auc) => println(f"$param, AUC=${auc * 100}%2.2f%%") }

0.001 L2 regularization parameter, AUC=66.55%
0.01 L2 regularization parameter, AUC=66.55%
0.1 L2 regularization parameter, AUC=66.63%
10.0 L2 regularization parameter, AUC=35.33%


In [80]:
import org.apache.spark.mllib.tree.impurity.Impurity
import org.apache.spark.mllib.tree.impurity.Entropy
import org.apache.spark.mllib.tree.impurity.Gini

def trainDTWithParams(input: RDD[LabeledPoint], maxDepth: Int, impurity: Impurity) = {
    DecisionTree.train(input, Algo.Classification, impurity, maxDepth)
}

In [81]:
val dtResultsEntropy = Seq(1,2,3,4,5,10,20).map{ param =>
    val model = trainDTWithParams(data, param,Entropy)
    val scoreAndLabels = data.map{ point =>
        val score = model.predict(point.features)
        (if (score > 0.5) 1.0 else 0.0, point.label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (s"$param tree depth", metrics.areaUnderROC)
}
dtResultsEntropy.foreach{case (param,auc) => println(f"$param, AUC = ${auc * 100}%2.2f%%")}

1 tree depth, AUC = 59.33%
2 tree depth, AUC = 61.68%
3 tree depth, AUC = 62.61%
4 tree depth, AUC = 63.63%
5 tree depth, AUC = 64.88%
10 tree depth, AUC = 76.26%
20 tree depth, AUC = 98.45%


In [83]:
def trainNBWithParams(input: RDD[LabeledPoint],lambda: Double) = {
    val nb = new NaiveBayes
    nb.setLambda(lambda)
    nb.run(input)
}
val nbResults = Seq(0.001,0.01,0.1,1.0,10.0).map{ param =>
    val model = trainNBWithParams(dataNB, param)
    val scoreAndLabels = dataNB.map{ point =>
        (model.predict(point.features), point.label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (s"$param lambda", metrics.areaUnderROC)
}
nbResults.foreach{ case(param, auc) => println(f"$param, AUC=${auc * 100}%2.2f%%")}

0.001 lambda, AUC=60.51%
0.01 lambda, AUC=60.51%
0.1 lambda, AUC=60.51%
1.0 lambda, AUC=60.51%
10.0 lambda, AUC=60.51%


In [84]:
val trainTestSplit = scaledDataCats.randomSplit(Array(0.6, 0.4), 123)
val train = trainTestSplit(0)
val test = trainTestSplit(1)

In [86]:
val regResultsTest = Seq(0.0, 0.001, 0.0025, 0.005, 0.01).map{ param =>
    val model = trainWithParams( train, param, numIterations, new SquaredL2Updater, 1.0)
    createMetrics(s"$param L2 regularization parameter", test, model)
}
regResultsTest.foreach{ case(param, auc) => println(f"$param, AUC = ${auc * 100}%2.6f%%")}

0.0 L2 regularization parameter, AUC = 67.173110%
0.001 L2 regularization parameter, AUC = 67.173110%
0.0025 L2 regularization parameter, AUC = 67.173110%
0.005 L2 regularization parameter, AUC = 67.140600%
0.01 L2 regularization parameter, AUC = 67.175986%
